REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [3]:
import pandas as pd

In [4]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [7]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [5]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [193]:
#Print the lenght of the df
print('There are',len(bus),'records')

There are 6315 records


#### Question 2b: How many unique business IDs are there?  

In [194]:
#Unique values from pandas function 'unique' 
n_IDs=len(pd.unique(bus['business_id']))
#Print the variable
print('There are', n_IDs, 'unique business IDs')

There are 6315 unique business IDs


#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [195]:
#Return a series containing counts of unique rows in the business_id column of the df with value_counts
bus['name'].value_counts().head(5)

STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
WALGREENS                     11
SAN FRANCISCO SOUP COMPANY    11
Name: name, dtype: int64

## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [196]:
#Checking the type of the second row of the zip code column
#Print the type
print('The zip codes values are stored as:')
type(bus['postal_code'].iloc[1])

The zip codes values are stored as:


str

#### Question 3b

What are the unique values of postal_code?

In [197]:
#Return length of unique value rows in the postal_code column of the business df 
unique_zips=len(pd.unique(bus['postal_code']))
#Print the variable
print('There are', unique_zips, 'unique zip code values')

There are 47 unique zip code values


#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [199]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [200]:
len(validZip)

24

In [201]:
#Create a new df where the validZip list is compared with the postal_codel column with isin
bus_valid = bus[bus['postal_code'].isin(validZip)]
#Print the lenght of the df
print('There are',len(bus_valid),'businesses in the new data frame')

There are 5999 businesses in the new data frame


## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [202]:
#Create a missing df where the null longitude values are stored, filtered with isnull
missing=bus_valid[pd.isnull(bus_valid['longitude'])]
#Print the lenght of the df
print(len(missing),'businesses are missing longitude values')

2483 businesses are missing longitude values


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [203]:
#Create a df that takes the missing df from 4a and fills the nulls with -1 value, then groups by postal_code column and counts
grouped_null = missing[['longitude', 'postal_code']].fillna(-1).groupby('postal_code').count() #Not the most elegant to count NaNs but works
#Create df from the bus_valid df with groupby and count
grouped_not_null=bus_valid[['longitude', 'postal_code']].groupby('postal_code').count()

In [204]:
#Create an empty df with a postal_code column
zipdf=pd.DataFrame(validZip,columns=['postal_code'])
#Merge the grouped_null and grouped_not_null into the empty df
nzip=zipdf.merge(grouped_null,left_on='postal_code',right_index=True,how='inner').merge(grouped_not_null,left_on='postal_code',right_index=True,how='inner')
#Renaming the columns
nzip.columns=['postal_code','null_lon','not_null_lon']
nzip

,postal_code,null_lon,not_null_lon
0,94102,196,262
1,94103,252,306
2,94104,59,74
3,94105,100,126
4,94107,247,204
5,94108,82,142
6,94109,146,237
7,94110,251,334
8,94111,115,167
9,94112,59,131


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [205]:
#Divide the columns of the df of missing and not missing long values
nzip['proportion']=nzip['null_lon']/(nzip['null_lon']+nzip['not_null_lon'])
#Sort the df, ascending false as we want descending
sorted_nzip=nzip.sort_values(by=['proportion'],ascending=False)
#Print the head, usually is always 5 but we can specify
sorted_nzip.head(5)

,postal_code,null_lon,not_null_lon,proportion
4,94107,247,204,0.547672
18,94124,88,76,0.536585
21,94132,65,69,0.485075
10,94114,108,122,0.469565
16,94122,123,147,0.455556


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [206]:
#Printing the length of the ins df
print('There are', len(ins),'rows in the inspection DataFrame')

There are 15430 rows in the inspection DataFrame


In [207]:
#Create a variable for the length of the unique values, filtered by business_id
unique_bus_ins=len(pd.unique(ins['business_id']))
#Prints the variable
print('There are', unique_bus_ins, 'unique businesses')

There are 5730 unique businesses


#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [214]:
print('Type takes the following values:')
#Create a df by selecting id and type to group by and then counting
type_count=ins[['business_id','type']].groupby('type').count()
type_count

Type takes the following values:


,business_id
type,
complaint,1
routine,15429


In [222]:
#n value occurrence
min = 10
#Value counter fro the 'type' on the ins df
vc_type = ins['type'].value_counts()
#Create ins2 data frame filtering by the value count greater than the n value
ins2 = ins[ins['type'].isin(vc_type[vc_type > min].index)]
#Create df for summary of counts, where the outliers are filtered
ins2_count=ins2[['business_id','type']].groupby('type').count()
ins2_count


,business_id
type,
routine,15429


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [224]:
#Made a copy of ins 2 bc of some warnings, from here down ins_2 is the same as ins2
ins2_year=ins2.copy()
#Changin the date column for the %Y%m%d format
ins2_year['date'] = pd.to_datetime(ins2_year['date'],format='%Y%m%d')
#We can the call dt year from the date column to extract the year 
ins2_year['year'] = pd.to_datetime(ins2_year['date']).dt.year
#Printing the values to check
ins2_year


,business_id,score,date,type,year
0,10,82,2016-05-03,routine,2016
1,10,94,2014-07-29,routine,2014
2,10,92,2014-01-14,routine,2014
3,19,94,2016-05-13,routine,2016
4,19,94,2014-11-10,routine,2014
...,...,...,...,...,...
15425,89515,98,2016-11-03,routine,2016
15426,89701,100,2016-12-06,routine,2016
15427,89790,96,2016-11-29,routine,2016
15428,89803,100,2016-12-15,routine,2016


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [227]:
range_years=ins2_year[['business_id','year']].groupby('year').count()
range_years

,business_id
year,
2013,38
2014,5629
2015,3923
2016,5839


In [229]:
#n value occurrence
min_repeat = 50
#Value counter for the 'year' 
vc_y = ins2_year['year'].value_counts()
#Create ins3 data frame filtering by the value count greater than the n value
ins3 = ins2_year[ins2_year['year'].isin(vc_y[vc_y > min_repeat].index)]
#Create df for summary of counts, where the outliers are filtered
nrange_years=ins3[['business_id','year']].groupby('year').count()
nrange_years

,business_id
year,
2014,5629
2015,3923
2016,5839


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [230]:
#Create a new df for the focus year
focus_year=ins3.loc[ins3['year']==2016]
#Prints the length of the focus year df
print('The 2016 focus year had',len(focus_year),'inspections')

The 2016 focus year had 5839 inspections


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [233]:
#Note: This merge takes into account the initial business data frame (not filtered by the ones with a valid zip code), hope that's okay
bus_ins=pd.merge(bus, focus_year, how="left", on=["business_id"])
bus_ins.head(16)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,2016-05-03,routine,2016.0
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,2016-05-13,routine,2016.0
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,2016-10-05,routine,2016.0
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,2016-03-11,routine,2016.0
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,2016-08-16,routine,2016.0
5,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,84.0,2016-06-14,routine,2016.0
6,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,78.0,2016-01-04,routine,2016.0
7,48,ART'S CAFE,747 IRVING ST,San Francisco,CA,94122,37.764013,-122.465749,+14156657440,94.0,2016-06-30,routine,2016.0
8,50,SUSHI ZONE,1815 MARKET ST.,San Francisco,CA,94103,37.771437,-122.423892,+14156211114,80.0,2016-06-14,routine,2016.0
9,54,RHODA GOLDMAN PLAZA,2180 POST ST,San Francisco,CA,94115,37.784626,-122.437734,+14153455060,NaN,NaT,NaN,NaN


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [234]:
#Create a new df with the nsmallest function with scores and only selecting the name, address and score columns
low_sc=bus_ins.nsmallest(20,['score']).loc[:,['name','address','score']]
low_sc


,name,address,score
4892,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD,52.0
4873,UNCLE CAFE,65 WAVERLY PL,55.0
6281,CRAZY PEPPER,2257 SAN JOSE AVE,55.0
7324,POKI TIME,2101 LOMBARD ST,55.0
3008,HING WANG BAKERY,339 JUDAH ST,56.0
5627,GOLDEN WOK,295 B ORIZABA AVE,56.0
549,MA'S DIM SUM & CAFE,1315 POWELL ST,57.0
836,RED A BAKERY,634 CLEMENT ST,57.0
4209,SAN TUNG RESTAURANT LLC,1031 IRVING ST,57.0
4903,"NEW GARDEN RESTAURANT, INC.",716 KEARNY ST,57.0


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!